In [22]:
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from dotenv import load_dotenv
import os

load_dotenv()

ELK_HOST = os.getenv("ELK_HOST")

es = Elasticsearch(hosts=[ELK_HOST])


In [6]:
index_list_1 = [
    "loadtest-webrtc-preliminary-2p-t3medium-final",
    "loadtest-webrtc-preliminary-3p-t3medium-final",
    "loadtest-webrtc-preliminary-5p-t3medium-final",
    "loadtest-webrtc-preliminary-6p-t3medium-final",
    "loadtest-webrtc-preliminary-7p-t3medium-final",
    "loadtest-webrtc-preliminary-8p-t3medium-final",
    "loadtest-webrtc-preliminary-10p-t3medium-final",
    "loadtest-webrtc-preliminary-3p-10s-t3medium-final",
    "loadtest-webrtc-preliminary-3p-40s-t3medium-final",
    "loadtest-webrtc-preliminary-2p-c5xlarge-final",
    "loadtest-webrtc-preliminary-3p-c5xlarge-final",
    "loadtest-webrtc-preliminary-5p-c5xlarge-final",
    "loadtest-webrtc-preliminary-6p-c5xlarge-final",
    "loadtest-webrtc-preliminary-7p-c5xlarge-final",
    "loadtest-webrtc-preliminary-8p-c5xlarge-final",
    "loadtest-webrtc-preliminary-10p-c5xlarge-final",
    "loadtest-webrtc-preliminary-3p-10s-c5xlarge-final",
    "loadtest-webrtc-preliminary-3p-40s-c5xlarge-final",
]

index_list_2 = [
    "loadtest-webrtc-preliminary-2p-t3medium-final-2",
    "loadtest-webrtc-preliminary-3p-t3medium-final-2",
    "loadtest-webrtc-preliminary-5p-t3medium-final-2",
    "loadtest-webrtc-preliminary-6p-t3medium-final-2",
    "loadtest-webrtc-preliminary-7p-t3medium-final-2",
    "loadtest-webrtc-preliminary-8p-t3medium-final-2",
    "loadtest-webrtc-preliminary-10p-t3medium-final-2",
    "loadtest-webrtc-preliminary-3p-10s-t3medium-final-2",
    "loadtest-webrtc-preliminary-3p-40s-t3medium-final-2",
    "loadtest-webrtc-preliminary-2p-c5xlarge-final-2",
    "loadtest-webrtc-preliminary-3p-c5xlarge-final-2",
    "loadtest-webrtc-preliminary-5p-c5xlarge-final-2",
    "loadtest-webrtc-preliminary-6p-c5xlarge-final-2",
    "loadtest-webrtc-preliminary-7p-c5xlarge-final-2",
    "loadtest-webrtc-preliminary-8p-c5xlarge-final-2",
    "loadtest-webrtc-preliminary-10p-c5xlarge-final-2",
    "loadtest-webrtc-preliminary-3p-10s-c5xlarge-final-2",
    "loadtest-webrtc-preliminary-3p-40s-c5xlarge-final-2",
]

index_list_3 = [
    "loadtest-webrtc-preliminary-2p-t3medium-final-3",
    "loadtest-webrtc-preliminary-3p-t3medium-final-3",
    "loadtest-webrtc-preliminary-5p-t3medium-final-3",
    "loadtest-webrtc-preliminary-6p-t3medium-final-3",
    "loadtest-webrtc-preliminary-7p-t3medium-final-3",
    "loadtest-webrtc-preliminary-8p-t3medium-final-3",
    "loadtest-webrtc-preliminary-10p-t3medium-final-3",
    "loadtest-webrtc-preliminary-3p-10s-t3medium-final-3",
    "loadtest-webrtc-preliminary-3p-40s-t3medium-final-3",
    "loadtest-webrtc-preliminary-2p-c5xlarge-final-3",
    "loadtest-webrtc-preliminary-3p-c5xlarge-final-3",
    "loadtest-webrtc-preliminary-5p-c5xlarge-final-3",
    "loadtest-webrtc-preliminary-6p-c5xlarge-final-3",
    "loadtest-webrtc-preliminary-7p-c5xlarge-final-3",
    "loadtest-webrtc-preliminary-8p-c5xlarge-final-3",
    "loadtest-webrtc-preliminary-10p-c5xlarge-final-3",
    "loadtest-webrtc-preliminary-3p-10s-c5xlarge-final-3",
    "loadtest-webrtc-preliminary-3p-40s-c5xlarge-final-3",
]

index_list = index_list_1 + index_list_2 + index_list_3

def get_qoe_data_from_elastic(index):
    # query: The elasticsearch query.
    query = {
        "query": {
            "exists": {
                "field": "vmaf"
            }
        }
    }
    # Scan function to get all the data.
    rel = scan(client=es,
               query=query,
               scroll='1m',
               index=index,
               raise_on_error=True,
               preserve_order=False,
               clear_scroll=True)
    # Keep response in a list.
    result = list(rel)
    temp = []
    # We need only '_source', which has all the fields required.
    # This elimantes the elasticsearch metdata like _id, _type, _index.
    for hit in result:
        temp.append(hit['_source'])
    # Create a dataframe.
    df = pd.DataFrame(temp)
    return df


In [33]:
from minio import Minio

client = Minio(
        os.getenv("MINIO_HOST"),
        access_key=os.getenv("MINIO_ACCESS_KEY"),
        secret_key=os.getenv("MINIO_SECRET_KEY"),
    )

found = client.bucket_exists("openvidu-loadtest-preliminary")
if not found:
    raise Exception("Bucket does not exist")

objects_names = []
objects = client.list_objects("openvidu-loadtest-preliminary", recursive=True)
for obj in objects:
    # objects_names.append({
    #     "name": obj.object_name[:-1],
    # })
    splitted = obj.object_name.encode('utf-8').split(b'/')
    objects_names.append({
        "test_type": splitted[0],
        "file": splitted[1]
    })

df_minio = pd.DataFrame(objects_names)
df_minio.to_csv(f"dfs/minio_items.csv", index=False)

In [19]:
df_list = list(map(get_qoe_data_from_elastic, index_list))

In [20]:
for i, df_user in enumerate(df_list):
    if df_user.empty:
        print(index_list[i])

loadtest-webrtc-preliminary-2p-t3medium-final
loadtest-webrtc-preliminary-3p-t3medium-final
loadtest-webrtc-preliminary-5p-t3medium-final
loadtest-webrtc-preliminary-6p-t3medium-final
loadtest-webrtc-preliminary-7p-t3medium-final
loadtest-webrtc-preliminary-8p-t3medium-final
loadtest-webrtc-preliminary-10p-t3medium-final
loadtest-webrtc-preliminary-3p-10s-t3medium-final
loadtest-webrtc-preliminary-3p-40s-t3medium-final
loadtest-webrtc-preliminary-2p-c5xlarge-final
loadtest-webrtc-preliminary-3p-c5xlarge-final
loadtest-webrtc-preliminary-5p-c5xlarge-final
loadtest-webrtc-preliminary-6p-c5xlarge-final
loadtest-webrtc-preliminary-7p-c5xlarge-final
loadtest-webrtc-preliminary-8p-c5xlarge-final
loadtest-webrtc-preliminary-10p-c5xlarge-final
loadtest-webrtc-preliminary-3p-10s-c5xlarge-final
loadtest-webrtc-preliminary-3p-40s-c5xlarge-final
loadtest-webrtc-preliminary-2p-t3medium-final-2
loadtest-webrtc-preliminary-3p-t3medium-final-2
loadtest-webrtc-preliminary-5p-t3medium-final-2
loadtest-w

In [16]:
for df_user in df_list:
    if not df_user.empty:
        df_user["@timestamp"] = pd.to_datetime(df_user["@timestamp"])

In [ ]:
os.makedirs("dfs", exist_ok=True)
for i, df in enumerate(df_list):
    if not df.empty:
        df.to_csv(f"dfs/{index_list[i]}.csv", index=False)


In [ ]:
node_types = ["browseremulator", "masternode", "medianode"]
for i, df_user in enumerate(df_list):
    min = df_user["@timestamp"].min()
    max = df_user["@timestamp"].max()
    for node_type in node_types:
        query = {
            "query": {
                "bool": {
                    "must": [
                        {
                            "match": {
                                "fields.node_role": node_type
                            }
                        },
                        {
                            "range": {
                                "@timestamp": {
                                    "lte": max.isoformat(),
                                    "gte": min.isoformat()
                                }
                            }
                        }
                    ]
                }
            }
        }
        rel = scan(client=es,
               query=query,
               scroll='1m',
               index="metric*",
               raise_on_error=True,
               preserve_order=False,
               clear_scroll=True)
        # Keep response in a list.
        result = list(rel)
        temp = []
        # We need only '_source', which has all the fields required.
        # This elimantes the elasticsearch metdata like _id, _type, _index.
        for hit in result:
            data = hit['_source']
            data_to_save = {
                "@timestamp": data["@timestamp"]
            }
            if "system" in data:
                data = data['system']
                if "cpu" in data:
                    data_to_save["cpu"] = data['cpu']['total']['norm']['pct']
                if "memory" in data:
                    data_to_save["memory"] = data['memory']['used']['pct']
            if ("cpu" in data) or ("memory" in data):
                temp.append(data_to_save)
        # Create a dataframe.
        df = pd.DataFrame(temp)
        if not df.empty:
            df = df.groupby("@timestamp", as_index=False).mean()
            df["@timestamp"] = pd.to_datetime(df["@timestamp"])
            df.to_csv(f"dfs/{index_list[i]}-{node_type}.csv", index=False)
